In [ ]:
!pip install kaggle

In [ ]:
import os
import json
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, GlobalMaxPooling1D, Attention, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.layers import Input, Embedding, Conv1D, LSTM, Dense, Flatten, MultiHeadAttention
from tensorflow.keras.models import Model

In [ ]:
kaggle_dictionary = json.load(open("kaggle.json"))

In [ ]:
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
    zip_ref.extractall()

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 54% 14.0M/25.7M [00:00<00:00, 143MB/s]
100% 25.7M/25.7M [00:00<00:00, 196MB/s]


In [ ]:
data = pd.read_csv("/content/IMDB Dataset.csv")
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

<ipython-input-5-dae63486fc9a>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


In [ ]:
data.shape

(50000, 2)

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)
Y_train = train_data["sentiment"].values
Y_test = test_data["sentiment"].values

In [ ]:
from tensorflow.keras.layers import Input, Embedding, Conv1D, LSTM, MultiHeadAttention, Dense, Flatten
from tensorflow.keras.models import Model

# Define the input
input_layer = Input(shape=(200,))
embedding_layer = Embedding(input_dim=5000, output_dim=128, input_length=200)(input_layer)
conv_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(embedding_layer)
lstm_layer = LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)(conv_layer)

# Apply MultiHeadAttention layer
attention_output = MultiHeadAttention(num_heads=2, key_dim=128)(lstm_layer, lstm_layer)
flatten_layer = Flatten()(attention_output)

# Output layer
output_layer = Dense(1, activation="sigmoid")(flatten_layer)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Model Summary
model.summary()

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 200)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, 200, 128)       │        640,000 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_2 (Conv1D)         │ (None, 198, 64)        │         24,640 │ embedding_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ (None, 198, 128)       │         98,816 │ conv1d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_2    │ (None, 198, 128)       │        131,968 │ lstm_2[0][0],          │
│ (MultiHeadAttention)      │                        │                │ lstm_2[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_2 (Flatten)       │ (None, 25344)          │              0 │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 1)              │         25,345 │ flatten_2[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 920,769 (3.51 MB)

 Trainable params: 920,769 (3.51 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 579s 1s/step - accuracy: 0.6831 - loss: 0.5262 - val_accuracy: 0.8871 - val_loss: 0.2737
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 615s 1s/step - accuracy: 0.9061 - loss: 0.2350 - val_accuracy: 0.8823 - val_loss: 0.2831
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 618s 1s/step - accuracy: 0.9257 - loss: 0.1902 - val_accuracy: 0.8827 - val_loss: 0.2843
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 559s 1s/step - accuracy: 0.9432 - loss: 0.1507 - val_accuracy: 0.8811 - val_loss: 0.3775
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 601s 1s/step - accuracy: 0.9571 - loss: 0.1123 - val_accuracy: 0.8754 - val_loss: 0.3841


In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 67s 213ms/step - accuracy: 0.8743 - loss: 0.3733
Test Loss: 0.36412784457206726
Test Accuracy: 0.8787999749183655


In [ ]:
# **Additional Metrics**
# Predict on the test data
Y_pred_probs = model.predict(X_test)
Y_pred = (Y_pred_probs > 0.5).astype("int32")

# Calculate precision, recall, and F1 score
precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Confusion matrix
conf_matrix = confusion_matrix(Y_test, Y_pred)
print("Confusion Matrix:")
print(conf_matrix)

313/313 ━━━━━━━━━━━━━━━━━━━━ 68s 217ms/step
Precision: 0.9054884509429963
Recall: 0.8479857114506847
F1 Score: 0.8757942201270752
Confusion Matrix:
[[4515  446]
 [ 766 4273]]


In [ ]:
def predict_sentiment(review):
    sequence = tokenizer.texts_to_sequences([review])
    padded_sequence = pad_sequences(sequence, maxlen=200)
    prediction = model.predict(padded_sequence)
    sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
    return sentiment

In [ ]:
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step
The sentiment of the review is: positive
